In [5]:
# Import library
import os
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [6]:
#创建转换器
train_transform = transforms.Compose([
    transforms.Resize((90, 90)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
test_transfrom = transforms.Compose([
    transforms.Resize((90, 90)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


In [7]:
#计算每个类别的图片数量
train_path = 'D:\Csci323\Emotion-domestic\Emotion-domestic\Train'
train_dataset = ImageFolder(root=train_path, transform=train_transform)
classes_name = train_dataset.classes
class_count = {}
for i in classes_name:
    class_path = os.path.join(train_path,i)
    image_count = len(os.listdir(class_path))
    class_count[i] = image_count
print(len(train_dataset))
print(class_count)

49601
{'Angry': 3422, 'Disgust': 3285, 'Fear': 1602, 'Happy': 24034, 'Neutral': 10908, 'Sad': 2875, 'Surprise': 3475}


In [8]:
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np

# 计算每个类别的权重
class_sample_counts = [3422, 3285, 1602, 24034, 10908, 2875, 3475]
weights = 1. / np.array(class_sample_counts)
samples_weights = np.array([weights[label] for _, label in train_dataset.samples])


# 创建采样器
sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights) * 2, replacement=True)

# 使用采样器创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)

In [9]:
test_dataset = ImageFolder(root='D:\Csci323\Emotion-domestic\Emotion-domestic\Test', transform=test_transfrom)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [10]:
# Build CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3) 
        self.activation1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=28224, out_features=512)
        self.fc2 = nn.Linear(512, 7)

    # forward propagation
    def forward(self,x):
        x = self.conv1(x) 
        x = self.activation1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.activation1(x)
        x = self.pool(x)
        x = self.flatten(x) 
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        return x 

In [11]:
# 启用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
#实例化模型
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
# 训练模型
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() 
    print(f'Epoch {epoch + 1} loss: {running_loss / 3100:.3f}')
# 测试模型
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images,labels = images.to(device),labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct / total:.2f}%')

print('Finished Training')

Epoch 1 loss: 1.376
Accuracy of the network on the test images: 65.26%
Epoch 2 loss: 0.950
Accuracy of the network on the test images: 71.44%
Epoch 3 loss: 0.709
Accuracy of the network on the test images: 78.14%
Epoch 4 loss: 0.558
Accuracy of the network on the test images: 78.64%
Epoch 5 loss: 0.456
Accuracy of the network on the test images: 82.76%
Epoch 6 loss: 0.373
Accuracy of the network on the test images: 82.06%
Epoch 7 loss: 0.306
Accuracy of the network on the test images: 83.22%
Epoch 8 loss: 0.260
Accuracy of the network on the test images: 84.98%
Epoch 9 loss: 0.222
Accuracy of the network on the test images: 86.52%
Epoch 10 loss: 0.186
Accuracy of the network on the test images: 85.84%
Epoch 11 loss: 0.165
Accuracy of the network on the test images: 86.24%
Epoch 12 loss: 0.142
Accuracy of the network on the test images: 86.42%
Epoch 13 loss: 0.125
Accuracy of the network on the test images: 86.54%
Epoch 14 loss: 0.119
Accuracy of the network on the test images: 85.64%
E

In [14]:
PATH = 'D:\Csci323\TT.pth'
torch.save(net, PATH)

In [28]:
from PIL import Image
path = 'D:\Csci323\Test3.jpg'
def test(path):
    classes_list = ['Angry','Disgust','Fear','Happy','Neutral','Neutral','Sad','Surprise']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = torch.load(PATH)
    net = net.to(device)
    image = Image.open(path)
    image = test_transfrom(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    outputs = net(image)
    _, predicted = torch.max(outputs.data, 1)
    print(classes_list[predicted.item()])
test(path)

Fear
